In [1]:
import np as np

import Ramsey_ExperimentV3
from tqdm import tqdm
import random

import estimator
import numpy as np

In [ ]:
fitted_data = []
T = 1 * np.pi
uni_delay = np.linspace(0, T, 20)
shots = 5000

uniform_local_G = []
n = 1
delay = [1]
fisher_delay = [0.4439410861020235, 1.784695781691183]

decay_fit = []

repetitions = 1000
# W =]
W = np.arange(0.001, 3.1, 0.1)
# np.delete(W, 0)
Decay = np.arange(0.001, 3.1, 0.1)
J = {}
J_list = [random.gauss(1, 0) for _ in range(n - 1)]
for rep in tqdm(range(repetitions), desc="Repetition"):
    rep_decay_fit = []
    for w in W:
        for d in Decay:
            # Create experiments
            batch_x_det, batch_y_det, batch_x_cross, batch_y_cross = Ramsey_ExperimentV3.ramsey_local(
                n, shots, delay, [d], [w], J
            )
            batch_det, batch_cross = Ramsey_ExperimentV3.ramsey_local_X(n, shots, fisher_delay, [d], [w], J)
            batch_x_det_uni, batch_y_det_uni, batch_x_cross_uni, batch_y_cross_uni = Ramsey_ExperimentV3.ramsey_local(
                n, shots, uni_delay, [d], [w], J
            )

            # Append batches and decay to decay fit list
            rep_decay_fit.append((
                w,d,
                (batch_x_det, batch_y_det, batch_x_cross, batch_y_cross),
                (batch_det, batch_cross),
                (batch_x_det_uni, batch_y_det_uni, batch_x_cross_uni, batch_y_cross_uni),
            ))

    decay_fit.append(rep_decay_fit)

Repetition:  79%|███████▉  | 794/1000 [15:44:51<5:03:28, 88.39s/it] 

In [ ]:

from importlib import reload
reload(estimator)

In [ ]:

for rep, rep_batches in enumerate(decay_fit):
    rep_fits = []
    print(f"Repetition {rep} / {repetitions}")
    for w, d,(batch_x_det, batch_y_det, batch_x_cross, batch_y_cross), (batch_det, batch_cross), (batch_x_det_uni, batch_y_det_uni, batch_x_cross_uni, batch_y_cross_uni) in rep_batches:
        Decay_local, W_local, J_local = estimator.one_by_one_fit(batch_x_det, batch_y_det, batch_x_cross, batch_y_cross)
        Decay_fisher, W_fisher, J_fisher = estimator.one_by_one_X(batch_det, batch_cross)
        Decay_uni, W_uni, J_uni = estimator.one_by_one_fit(batch_x_det_uni, batch_y_det_uni, batch_x_cross_uni, batch_y_cross_uni)

        rep_fits.append((
            w,d,
            (Decay_local, W_local, J_local),
            (Decay_fisher, W_fisher, J_fisher),
            (Decay_uni, W_uni, J_uni),
        ))

    fitted_data.append(rep_fits)

In [ ]:
from importlib import reload
reload(estimator)
errors_fisher = []
errors_local = []
uniform_local = []
for rep, rep_fits in enumerate(fitted_data):
    rep_error_fisher = []
    rep_error_local = []
    rep_error_uniform = []

    for w,d, (Decay_local, W_local, J_local), (Decay_fisher, W_fisher, J_fisher), (Decay_uni, W_uni, J_uni) in rep_fits:
        fisher_errors = estimator.percent_error(
            np.concatenate([[w], [d], J_list]),
            np.concatenate([W_fisher, Decay_fisher, J_fisher])
        )
        local_errors = estimator.percent_error(
            np.concatenate([[w], [d], J_list]),
            np.concatenate([W_local, Decay_local, J_local])
        )

        uniform_errors = estimator.percent_error(
            np.concatenate([[w], [d], J_list]),
            np.concatenate([W_uni, Decay_uni, J_uni])
        )

        rep_error_fisher.append(fisher_errors)
        rep_error_local.append(local_errors)
        rep_error_uniform.append(uniform_errors)

    errors_fisher.append(rep_error_fisher)
    errors_local.append(rep_error_local)
    uniform_local.append(rep_error_uniform)



In [ ]:
def reshape_2d_array(array):
    array = np.array(array)
    new_array = []
    for a in array:
        new_array.append(a.reshape(int(np.sqrt(len(a))), int(np.sqrt(len(a))),2))
    new_array = np.array(new_array).transpose()
    return new_array
errors_fisher_reshaped = reshape_2d_array(errors_fisher)
errors_local_reshaped = reshape_2d_array(errors_local)
errors_uniform_reshaped = reshape_2d_array(uniform_local)



In [ ]:
k = 10

# errors_local_reshaped = list(map(list, zip(*errors_local)))
# errors_uniform_reshaped = list(map(list, zip(*uniform_local)))
# errors_fisher_reshaped_G = list(map(list, zip(*uniform_local_G)))

errors_fisher_reshaped_a = errors_fisher_reshaped[1]
errors_local_reshaped_a = errors_local_reshaped[1]
errors_uniform_reshaped_a = errors_uniform_reshaped[1]

errors_fisher_reshaped_w = errors_fisher_reshaped[0]
errors_local_reshaped_w = errors_local_reshaped[0]
errors_uniform_reshaped_w = errors_uniform_reshaped[0]


errors_fisher_median_a, errors_fisher_std_a = np.zeros((len(W),len(W))),np.zeros((len(W),len(W)))
errors_local_median_a, errors_local_std_a = np.zeros((len(W),len(W))),np.zeros((len(W),len(W)))
errors_uniform_median_a, errors_uniform_std_a = np.zeros((len(W),len(W))),np.zeros((len(W),len(W)))

errors_fisher_median_w, errors_fisher_std_w = np.zeros((len(W),len(W))),np.zeros((len(W),len(W)))
errors_local_median_w, errors_local_std_w = np.zeros((len(W),len(W))),np.zeros((len(W),len(W)))
errors_uniform_median_w, errors_uniform_std_w = np.zeros((len(W),len(W))),np.zeros((len(W),len(W)))

for i in range(len(W)):
        errors_fisher_median_a[i], errors_fisher_std_a[i] = estimator.mean_of_medians(errors_fisher_reshaped_a[i, :], k)
        errors_local_median_a[i], errors_local_std_a[i] = estimator.mean_of_medians(errors_local_reshaped_a[i, :], k)
        errors_uniform_median_a[i], errors_uniform_std_a[i] = estimator.mean_of_medians(errors_uniform_reshaped_a[i, :], k)

        errors_fisher_median_w[i], errors_fisher_std_w[i] = estimator.mean_of_medians(errors_fisher_reshaped_w[i, :], k)
        errors_local_median_w[i], errors_local_std_w[i] = estimator.mean_of_medians(errors_local_reshaped_w[i, :], k)
        errors_uniform_median_w[i], errors_uniform_std_w[i] = estimator.mean_of_medians(errors_uniform_reshaped_w[i, :], k)



In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 4))  # Adjust the figure size to make the plot longer
plt.rcParams['text.usetex'] = True
font = {'family': 'normal',
        'weight': 'bold',
        'size': 14}

plt.rc('font', **font)

plt.errorbar(W, np.array(errors_uniform_median_a[:,4]), yerr= np.array(errors_uniform_std_a[:,4]),
             fmt='o', color='blue')
plt.errorbar(W, np.array(errors_fisher_median_a[:,4]), yerr= np.array(errors_fisher_std_a[:,4]),
             fmt='o', color='red')
plt.errorbar(W, np.array(errors_local_median_a[:,4]), yerr= np.array(errors_local_std_a[:,4]), fmt='o',
             color='green')


def rao_bound_complex(T, a, sigma):
    I = 0
    for t in T:
        I += 1 / (sigma ** 2) * t ** 2 * np.exp(-2 * t * a)
    bound = 1 / np.sqrt(I)
    return bound

sigma_single = 1 / np.sqrt(shots/2)
Gamma = np.linspace(0, 4, 100)
error_single = []
for g in Gamma:
    error_single.append(rao_bound_complex([1], g, sigma_single))

plt.plot(Gamma, error_single, label='Single-shot bound', color='black')

plt.title("Dephasing")
# plt.title(f'Shots={Decay} ')
# plt.xlabel('$\omega')
# plt.ylabel('mean / $\gamma$')
plt.grid()
# plt.yscale('log')
# plt.xscale('log')
# plt.xlim([0, 2])
plt.ylim([0, 0.5])

plt.grid()
plt.legend()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 4))  # Adjust the figure size to make the plot longer
plt.rcParams['text.usetex'] = True
font = {'family': 'normal',
        'weight': 'bold',
        'size': 14}

plt.rc('font', **font)

plt.errorbar(W,  np.array(errors_uniform_median_w[:,0]), yerr= np.array(errors_uniform_std_w[:,4]),
             fmt='o', color='blue')
plt.errorbar(W,  np.array(errors_fisher_median_w[:,0]), yerr= np.array(errors_fisher_std_w[:,4]),
             fmt='o', color='red')
plt.errorbar(W,  np.array(errors_local_median_w[:,0]), yerr= np.array(errors_local_std_w[:,4]), fmt='o',
             color='green')
# plt.errorbar(Decay, 100 * np.array(errors_uniform_median_G/Decay ), yerr=100 * np.array(errors_uniform_std_G/Decay ), fmt='o',
#              color='magenta')
# Scatter points for 75th percentiles
plt.title("Freq")

# plt.title(f'Shots={Decay} ')
# plt.xlabel('$\omega')
# plt.ylabel('mean / $\omega$')
plt.grid()
# plt.yscale('log')
# plt.xscale('log')
# plt.xlim([0, 2])
# plt.ylim([0,50])
sigma_single = 1 / np.sqrt(shots/2)
Gamma = np.linspace(0, 4, 100)
error_single = []
for g in Gamma:
    error_single.append(rao_bound_complex([1], g, sigma_single))

plt.plot(Gamma, error_single, label='Single-shot bound', color='black')
plt.ylim([0, 1])
plt.grid()

plt.legend()

In [ ]:
#Heatmap
# Plotting the heatmap
# sqrt_decay_w_array = np.zeros((len(Decay), len(W)))
# for i in range(len(Decay)):
#     for j in range(len(W)):
#         sqrt_decay_w_array[i, j] = np.sqrt(Decay[i]**2 + W[j]**2)
plt.figure(figsize=(8, 6))
plt.imshow(errors_fisher_median_a, aspect='auto', origin='lower',
           extent=[W.min(), W.max(), Decay.min(), Decay.max()], cmap='jet')
plt.colorbar(label='Mean Error (Fisher)')
plt.xlabel('W')
plt.ylabel('Decay')
plt.title('Heatmap of Median Errors (Fisher)')
plt.show()

plt.figure(figsize=(8, 6))
plt.imshow(errors_local_median_a, aspect='auto', origin='lower',
           extent=[W.min(), W.max(), Decay.min(), Decay.max()], cmap='jet')
plt.colorbar(label='Mean Error (local)')
plt.xlabel('W')
plt.ylabel('Decay')
plt.title('Heatmap of Median Errors (local)')
plt.show()


plt.figure(figsize=(8, 6))
plt.imshow(errors_uniform_median_a, aspect='auto', origin='lower',
           extent=[W.min(), W.max(), Decay.min(), Decay.max()], cmap='jet')
plt.colorbar(label='Median Error (Uniform)')
plt.xlabel('W')
plt.ylabel('Decay')
plt.title('Heatmap of Median Errors (Uniform)')
plt.show()


In [ ]:
fisher_a = []
local_a = []
uniform_a = []

fisher_w = []
local_w = []
uniform_w = []

for rep, rep_fits in enumerate(fitted_data):
    # Initialize lists for this repetition
    rep_fisher_a = []
    rep_local_a = []
    rep_uniform_a = []

    rep_fisher_w = []
    rep_local_w = []
    rep_uniform_w = []

    for w,d, (Decay_local, W_local, J_local), (Decay_fisher, W_fisher, J_fisher), (Decay_uni, W_uni, J_uni) in rep_fits:
        # Append decays
        rep_fisher_a.append(Decay_fisher)
        rep_local_a.append(Decay_local)
        rep_uniform_a.append(Decay_uni)

        # Append detunings
        rep_fisher_w.append(W_fisher)
        rep_local_w.append(W_local)
        rep_uniform_w.append(W_uni)

    fisher_a.append(rep_fisher_a)
    local_a.append(rep_local_a)
    uniform_a.append(rep_uniform_a)

    fisher_w.append(rep_fisher_w)
    local_w.append(rep_local_w)
    uniform_w.append(rep_uniform_w)



fisher_a = np.array(fisher_a).transpose()[0]
local_a = np.array(local_a).transpose()[0]
uniform_a = np.array(uniform_a).transpose()[0]



fisher_w= np.array(fisher_w).transpose()[0]
local_w = np.array(local_w).transpose()[0]
uniform_w = np.array(uniform_w).transpose()[0]


fisher_a_mean = [np.mean(fisher_a[i]) for i in range(len(W)**2)]
fisher_a_std = [np.std(fisher_a[i]) for i in range(len(W)**2)]

local_a_mean = [np.mean(local_a[i]) for i in range(len(W)**2)]
local_a_std = [np.std(local_a[i])  for i in range(len(W)**2)]

uniform_a_mean = [np.mean(uniform_a[i]) for i in range(len(W)**2)]
uniform_a_std = [np.std(uniform_a[i])  for i in range(len(W)**2)]


fisher_w_mean = [np.mean(fisher_w[i]) for i in range(len(W)**2)]
fisher_w_std = [np.std(fisher_w[i])  for i in range(len(W)**2)]

local_w_mean = [np.mean(local_w[i]) for i in range(len(W)**2)]
local_w_std = [np.std(local_w[i])  for i in range(len(W)**2)]

uniform_w_mean = [np.mean(uniform_w[i]) for i in range(len(W)**2)]
uniform_w_std = [np.std(uniform_w[i]) for i in range(len(W)**2)]
#

def bias_matrix_a(matrix, true):
    B = np.zeros((len(W), len(W)))
    for i in range(len(W)):
        for j in range(len(W)):
            B[i, j] = matrix[i, j] - true[j]
    return np.array(B)

def bias_matrix_w(matrix, true):
    B = np.zeros((len(W), len(W)))
    for i in range(len(W)):
        for j in range(len(W)):
            B[i, j] = matrix[i, j] - true[i]
    return np.array(B)
# print("Fisher_a: ", len(fisher_a))

In [ ]:
fisher_resaped = np.array(fisher_a_mean).reshape((len(W),len(Decay)))
plt.plot(Decay,fisher_resaped[4,:])
plt.show()
fisher_resaped = np.array(fisher_w_mean).reshape((len(W),len(Decay)))
plt.plot(W,fisher_resaped[:,1])

In [ ]:
# Fisher
res = np.array(fisher_w_mean).reshape((len(W), len(Decay)))
bias = bias_matrix_w(res, Decay)
plt.figure(figsize=(8, 6))
plt.imshow(bias, aspect='auto', origin='lower',
           extent=[W.min(), W.max(), Decay.min(), Decay.max()],
           cmap='jet', vmin=-3, vmax=3)
plt.colorbar(label='Mean Error (Fisher)')
plt.xlabel('W')
plt.ylabel('Decay')
plt.title('Heatmap of Median Errors (Fisher)')
plt.show()

# Local
res = np.array(local_w_mean).reshape((len(W), len(Decay)))
bias = bias_matrix_w(res, Decay)
plt.figure(figsize=(8, 6))
plt.imshow(bias, aspect='auto', origin='lower',
           extent=[W.min(), W.max(), Decay.min(), Decay.max()],
           cmap='jet', vmin=-3, vmax=3)
plt.colorbar(label='Mean Error (Local)')
plt.xlabel('W')
plt.ylabel('Decay')
plt.title('Heatmap of Median Errors (Local)')
plt.show()

# Uniform
res = np.array(uniform_w_mean).reshape((len(W), len(Decay)))
bias = bias_matrix_w(res, Decay)
plt.figure(figsize=(8, 6))
plt.imshow(bias, aspect='auto', origin='lower',
           extent=[W.min(), W.max(), Decay.min(), Decay.max()],
           cmap='jet', vmin=-3, vmax=3)
plt.colorbar(label='Mean Error (Uniform)')
plt.xlabel('W')
plt.ylabel('Decay')
plt.title('Heatmap of Median Errors (Uniform)')
plt.show()


In [ ]:
# Fisher
res = np.array(fisher_a_mean).reshape((len(W), len(Decay)))
bias = bias_matrix_a(res, Decay)
plt.figure(figsize=(8, 6))
plt.imshow(bias, aspect='auto', origin='lower',
           extent=[W.min(), W.max(), Decay.min(), Decay.max()],
           cmap='jet', vmin=-3, vmax=3)
plt.colorbar(label='Mean Error (Fisher)')
plt.xlabel('W')
plt.ylabel('Decay')
plt.title('Heatmap of Median Errors (Fisher)')
plt.show()

# Local
res = np.array(local_a_mean).reshape((len(W), len(Decay)))
bias = bias_matrix_a(res, Decay)
plt.figure(figsize=(8, 6))
plt.imshow(bias, aspect='auto', origin='lower',
           extent=[W.min(), W.max(), Decay.min(), Decay.max()],
           cmap='jet', vmin=-3, vmax=3)
plt.colorbar(label='Mean Error (Local)')
plt.xlabel('W')
plt.ylabel('Decay')
plt.title('Heatmap of Median Errors (Local)')
plt.show()

# Uniform
res = np.array(uniform_a_mean).reshape((len(W), len(Decay)))
bias = bias_matrix_a(res, Decay)
plt.figure(figsize=(8, 6))
plt.imshow(bias, aspect='auto', origin='lower',
           extent=[W.min(), W.max(), Decay.min(), Decay.max()],
           cmap='jet', vmin=-3, vmax=3)
plt.colorbar(label='Mean Error (Uniform)')
plt.xlabel('W')
plt.ylabel('Decay')
plt.title('Heatmap of Median Errors (Uniform)')
plt.show()


In [ ]:
res = np.array(fisher_w_mean).reshape((len(W), len(Decay)))
bias = bias_matrix_w(res, Decay)
plt.plot(Decay, bias[5, :])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Fisher
res = np.array(fisher_w_mean).reshape((len(W), len(Decay)))
bias = bias_matrix_w(res, Decay)
W_grid, Decay_grid = np.meshgrid(W, Decay)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
surface = ax.plot_surface(W_grid, Decay_grid, bias.T, cmap='jet', edgecolor='none')
fig.colorbar(surface, ax=ax, label='Mean Error (Fisher)')
ax.set_xlabel('W')
ax.set_ylabel('Decay')
ax.set_zlabel('Mean Error')
ax.set_title('3D Surface Plot of Median Errors (Fisher)')
plt.show()

# Local
res = np.array(local_w_mean).reshape((len(W), len(Decay)))
bias = bias_matrix_w(res, Decay)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
surface = ax.plot_surface(W_grid, Decay_grid, bias.T, cmap='jet', edgecolor='none')
fig.colorbar(surface, ax=ax, label='Mean Error (Local)')
ax.set_xlabel('W')
ax.set_ylabel('Decay')
ax.set_zlabel('Mean Error')
ax.set_title('3D Surface Plot of Median Errors (Local)')
plt.show()

# Uniform
res = np.array(uniform_w_mean).reshape((len(W), len(Decay)))
bias = bias_matrix_w(res, Decay)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
surface = ax.plot_surface(W_grid, Decay_grid, bias.T, cmap='jet', edgecolor='none')
fig.colorbar(surface, ax=ax, label='Mean Error (Uniform)')
ax.set_xlabel('W')
ax.set_ylabel('Decay')
ax.set_zlabel('Mean Error')
ax.set_title('3D Surface Plot of Median Errors (Uniform)')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Fisher
res = np.array(fisher_a_mean).reshape((len(W), len(Decay)))
bias = bias_matrix_a(res, Decay)
W_grid, Decay_grid = np.meshgrid(W, Decay)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
surface = ax.plot_surface(W_grid, Decay_grid, bias.T, cmap='jet', edgecolor='none')
fig.colorbar(surface, ax=ax, label='Mean Error (Fisher)')
ax.set_xlabel('W')
ax.set_ylabel('Decay')
ax.set_zlabel('Mean Error')
ax.set_title('3D Surface Plot of Median Errors (Fisher)')
plt.show()

# Local
res = np.array(local_a_mean).reshape((len(W), len(Decay)))
bias = bias_matrix_a(res, Decay)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
surface = ax.plot_surface(W_grid, Decay_grid, bias.T, cmap='jet', edgecolor='none')
fig.colorbar(surface, ax=ax, label='Mean Error (Local)')
ax.set_xlabel('W')
ax.set_ylabel('Decay')
ax.set_zlabel('Mean Error')
ax.set_title('3D Surface Plot of Median Errors (Local)')
plt.show()

# Uniform
res = np.array(uniform_a_mean).reshape((len(W), len(Decay)))
bias = bias_matrix_a(res, Decay)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
surface = ax.plot_surface(W_grid, Decay_grid, bias.T, cmap='jet', edgecolor='none')
fig.colorbar(surface, ax=ax, label='Mean Error (Uniform)')
ax.set_xlabel('W')
ax.set_ylabel('Decay')
ax.set_zlabel('Mean Error')
ax.set_title('3D Surface Plot of Median Errors (Uniform)')
plt.show()


In [ ]:
from tqdm import tqdm
import random

T = 1 * np.pi
shots = 1000
errors_fisher = []
errors_local = []
uniform_local = []
uniform_local_G = []
n = 1
delay = [1]
fisher_delay = [0.4439410861020235, 1.784695781691183]

decay_fit = []

repetitions = 100
# W =]
Decay = np.arange(0.001, 4, 0.25)
# np.delete(W, 0)
W = [1]
J = {}
J_list = [random.gauss(1, 0) for _ in range(n - 1)]

for rep in range(repetitions):
    rep_Decay_fisher = []

    rep_decay_fit = []
    for decay in tqdm(Decay, desc=f"Repetition {rep}"):
        batch_x_det, batch_y_det, batch_x_cross, batch_y_cross = Ramsey_ExperimentV3.ramsey_local(n, shots, delay,
                                                                                                  [decay],
                                                                                                  W, J)
        Decay_local, W_local, J_local = estimator.one_by_one_fit(batch_x_det, batch_y_det, batch_x_cross,
                                                                 batch_y_cross)
        # batch_det, batch_cross = Ramsey_ExperimentV3.ramsey_local_X(n, shots, fisher_delay, [decay], W, J)
        # Decay_fisher, W_fisher, J_fisher = estimator.one_by_one_X(batch_det, batch_cross)

        rep_Decay_fisher.append(Decay_local)
    decay_fit.append(rep_Decay_fisher)



In [ ]:
import matplotlib.pyplot as plt
decay_fit_t = np.array(decay_fit).transpose()[0]
decay_fit_t_mean = [np.mean(decay_fit_t[i]) for i in range(len(Decay))]
decay_fit_t_std = [np.std(decay_fit_t[i]) / np.sqrt(repetitions) for i in range(len(Decay))]

# plt.plot(Decay,decay_fit_t_mean,label='fit')
plt.plot(Decay, Decay)
plt.errorbar(Decay, decay_fit_t_mean, yerr=decay_fit_t_std, label='std')
plt.legend()

In [ ]:
for w in W:
    for d in Decay:


In [ ]:
import pandas as pd
import os
home = user_home = os.path.expanduser("~")
graphs = user_home+ r'\OneDrive - Bar-Ilan University - Students\Data\Crosstalk\graphs_for_paper\Bias'

    ########################################
df_fisher_median_a  = pd.DataFrame(fisher_a_mean)
df_fisher_std_a      = pd.DataFrame(fisher_a_std)
df_local_median_a    = pd.DataFrame(local_a_mean)
df_local_std_a       = pd.DataFrame(local_a_std)
df_uniform_median_a  = pd.DataFrame(uniform_a_mean)
df_uniform_std_a     = pd.DataFrame(uniform_a_std)

df_fisher_median_w   = pd.DataFrame(fisher_w_mean)
df_fisher_std_w       = pd.DataFrame(fisher_w_std)
df_local_median_w     = pd.DataFrame(local_w_mean)
df_local_std_w        = pd.DataFrame(local_w_std)
df_uniform_median_w   = pd.DataFrame(uniform_w_mean)
df_uniform_std_w      = pd.DataFrame(uniform_w_std)

########################################
# 3. Optionally label rows and columns with W
#    (Uncomment if you have a W array and want labeled CSVs)
########################################
# df_fisher_median_a.index = W
# df_fisher_median_a.columns = W
# df_fisher_std_a.index = W
# df_fisher_std_a.columns = W
# ... repeat for all DataFrames ...

########################################
# 4. Save DataFrames to CSV
########################################
df_fisher_median_a.to_csv(graphs+ "errors_fisher_median_a.csv", index=False)
df_fisher_std_a.to_csv(graphs + "errors_fisher_std_a.csv", index=False)
df_local_median_a.to_csv(graphs + "errors_local_median_a.csv", index=False)
df_local_std_a.to_csv(graphs + "errors_local_std_a.csv", index=False)
df_uniform_median_a.to_csv(graphs + "errors_uniform_median_a.csv", index=False)
df_uniform_std_a.to_csv( graphs + "errors_uniform_std_a.csv", index=False)

df_fisher_median_w.to_csv(graphs + "errors_fisher_median_w.csv", index=False)
df_fisher_std_w.to_csv(graphs + "errors_fisher_std_w.csv", index=False)
df_local_median_w.to_csv(graphs + "errors_local_median_w.csv", index=False)
df_local_std_w.to_csv(graphs + "errors_local_std_w.csv", index=False)
df_uniform_median_w.to_csv(graphs + "errors_uniform_median_w.csv", index=False)
df_uniform_std_w.to_csv(graphs + "errors_uniform_std_w.csv", index=False)

print("CSV files saved successfully!")


In [82]:
import pandas as pd
import os

# Define the directory where the CSV files are saved
home = os.path.expanduser("~")
graphs = os.path.join(home, r'OneDrive - Bar-Ilan University - Students\Data\Crosstalk\graphs_for_paper\Bias')

# Load the CSV files into DataFrames
df_fisher_median_a = pd.read_csv(os.path.join(graphs, "errors_fisher_median_a.csv"))
df_fisher_std_a = pd.read_csv(os.path.join(graphs, "errors_fisher_std_a.csv"))
df_local_median_a = pd.read_csv(os.path.join(graphs, "errors_local_median_a.csv"))
df_local_std_a = pd.read_csv(os.path.join(graphs, "errors_local_std_a.csv"))
df_uniform_median_a = pd.read_csv(os.path.join(graphs, "errors_uniform_median_a.csv"))
df_uniform_std_a = pd.read_csv(os.path.join(graphs, "errors_uniform_std_a.csv"))

df_fisher_median_w = pd.read_csv(os.path.join(graphs, "errors_fisher_median_w.csv"))
df_fisher_std_w = pd.read_csv(os.path.join(graphs, "errors_fisher_std_w.csv"))
df_local_median_w = pd.read_csv(os.path.join(graphs, "errors_local_median_w.csv"))
df_local_std_w = pd.read_csv(os.path.join(graphs, "errors_local_std_w.csv"))
df_uniform_median_w = pd.read_csv(os.path.join(graphs, "errors_uniform_median_w.csv"))
df_uniform_std_w = pd.read_csv(os.path.join(graphs, "errors_uniform_std_w.csv"))

# Print or inspect one of the DataFrames
print(df_fisher_median_a.head())  # Displays the first few rows


          0
0  0.000511
1  0.242751
2  0.490413
3  0.732201
4  0.976056
